In [ ]:
%pip install yfinance numpy scipy
import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime


In [ ]:
# Black-Scholes Greeks
def calculate_greeks(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == 'call':
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) - r * K * np.exp(-r * T) * norm.cdf(d2)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == 'put':
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) + r * K * np.exp(-r * T) * norm.cdf(-d2)
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("Option type must be 'call' or 'put'")

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)

    return {
        'Delta': delta,
        'Gamma': gamma,
        'Theta': theta,
        'Vega': vega / 100,   # scaled for 1% change
        'Rho': rho / 100      # scaled for 1% change
    }

# Get user input
ticker = input("Enter a stock ticker: ").upper()
option_type = input("Option type (call or put): ").lower()

stock = yf.Ticker(ticker)
spot_price = stock.history(period="1d")['Close'].iloc[-1]

# Show available expiration dates
options = stock.options
print(f"Available expiration dates: {options}")
exp_date = input("Pick expiration date from above: ")

# Choose a strike
option_chain = stock.option_chain(exp_date)
chain = option_chain.calls if option_type == 'call' else option_chain.puts
print(chain[['strike', 'lastPrice', 'impliedVolatility']].head())

strike = float(input("Pick a strike from the list above: "))
option_row = chain[chain['strike'] == strike].iloc[0]

# Inputs for BSM
K = strike
S = spot_price
T = (datetime.strptime(exp_date, "%Y-%m-%d") - datetime.today()).days / 365
r = 0.05  # risk-free rate (can be dynamic)
sigma = option_row['impliedVolatility']

# Calculate Greeks
greeks = calculate_greeks(S, K, T, r, sigma, option_type)
print(f"\nGreeks for {ticker} {option_type.upper()} option (Strike: {K}, Exp: {exp_date})")
for greek, value in greeks.items():
    print(f"{greek}: {value:.4f}")
